# CNNを用いた坂道グループのメンバーの顔分類

ディープラーニングをやりたくて、さわりとしてCNNを使った顔分類をやりました。  
  
乃木坂メンバーの顔をCNNで分類  
https://qiita.com/nirs_kd56/items/bc78bf2c3164a6da1ded2    
いろいろなサイトを見た結果、こちらのサイトが丁寧でわかりやすかったのでかなり参考にさせてもらいました。  
  
テーマが似たような感じなのはたまたま趣味が同じだったってことで、ここを変えたとこで差別化はできないと思ったのでこんなかんじになりました。  



## 環境
windows10　バージョン1903  
Python 3.7.3

## 画像のスクレイピング
参考記事のスクレイピングは、chromeの仕様変更に伴って使えなくなっていたようなので、一から作り直しました。 
  
画像読み込みがページ切り替えで分かりやすかったYahoo画像検索を使用しました。  
一度の実行で学習させたい人数分を一度にスクレイピングすることが出来ます。  
  
  
検索結果画面のサムネイルを保存しているような形なので、画像サイズがかなり小さくなっていますが、一応使えます。
``` python:image_collect.py
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import urllib
import time
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


def main(search_word,maxcount):
    # 検索URL準備
    load_url="https://search.yahoo.co.jp/image/search?p="+search_word+"&ei=UTF-8&b="


    # 保存用フォルダを作る
    data = Path("./data")
    data.mkdir(exist_ok=True)
    out_folder=Path("data/"+search_word)
    out_folder.mkdir(exist_ok=True)

    # すべてのhtmlタグを検索し、リンクを取得する
    count=1

    for i in range(1,maxcount,20):

        # ページ切り替え
        html = requests.get(load_url+str(i))
        soup = BeautifulSoup(html.content, "html.parser")

        for element in soup.find_all("img"):
            src=element.get("src")

            # 絶対URLから画像を取得する
            image_url=urllib.parse.urljoin(load_url,src)
            imgdata=requests.get(image_url)

            # URLから最後のファイル名を取り出して保存先のファイル名とつなげる
            filename=str(count)+".png"
            out_path=out_folder.joinpath(filename)

            # 画像データをファイルに書き出す
            with open(out_path,mode="wb") as f:
                f.write(imgdata.content)

            # 枚数が十分だったらやめる
            if maxcount<=count:
                print(search_word+"完了")
                break
            else:
                count+=1


            # 0.2秒待つ
            time.sleep(0.2)
        print(str(count) + "枚取得")

if __name__ == '__main__':
    member = []
    num = int(input("人数"))
    for i in range(num):
        member.append(input("検索ワード"))
    b = int(input("取得枚数"))
    for i in range(len(member)):
        main(member[i], b)
```
  
このコードを実行すると、標準入力が求められるので
1. 人数
2. 検索ワード(×人数)
3. 取得枚数  
![入力したときの感じ](./1.png)


↑のように入力していきます。  
  
スクレイピングした画像はおなじ階層に以下のようなフォルダ構成で保存されます。
>data/  
>├ 検索ワード1/  
>  │　       　　├ 1.png  
>  │　       　　├ 2.png  
>  │　       　　├ ...  
>  │　       
>├ 検索ワード2/  
>  │　       　　├ 1.png  
>  │　       　　├ 2.png  
>  │　       　　├ ...  　       
>...
  
600～700枚が限界みたいです。  
実際200～300枚で十分学習はできるうえ、画像が多いとこの後の作業がだいぶきつくなるのでほどほどにしておきましょう。

## 顔部分のトリミング
参考記事のコードを少し変えました。  
  
  
スクレイピングで作成されるフォルダ名に、検索ワードがそのまま使われるようになっているので、日本語のパスだとエラーが出るopencvの関数の対策などをしました。  
  
また、人数分コードを変えてトリミングするのも面倒なので、Tkinterを使用してGUIでフォルダを指定し、画像フォルダがまとめて保存されているフォルダ（今回はdataフォルダ）を選択すればすべてにトリミングが適用されるようにしました。  
この変更はここから学習のコードまですべてにしています。
  
  
このへんもパス関連で不具合が多く、Jupyterではうまく動かないというところが解決できませんでした。
``` python:face_detect.py
import glob
import os
import tkinter
from tkinter import filedialog
import cv2
import numpy as np
import sys

"""
dataディレクトリから画像を読み込んで顔を切り取ってfaceディレクトリに保存.
"""
out_dir = "./face"

# フォルダ指定
def dirdialog_clicked():
    root = tkinter.Tk()
    root.withdraw()
    iDir = os.path.abspath(os.path.dirname(__file__))
    iDirPath = filedialog.askdirectory(initialdir = iDir)
    root.destroy()
    return iDirPath


# パスに日本語が含まれる場合の対策
# np.profileとcv2.imdecodeに分解した
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None
#cv2.imencode + np.ndarray.tofile に分解
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)

        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False


if __name__ == "__main__":
    #各フォルダ指定
    image_dir = dirdialog_clicked()
    img_lists=os.listdir(path=image_dir)
    out_dir="."

    for lists in range(len(img_lists)):
        save_num = 0
        # 元画像を取り出して顔部分を正方形で囲み、64×64pにリサイズ、別のファイルにどんどん入れてく
        in_dir = image_dir +"/"+img_lists[lists] + "/*.png"
        print(in_dir)
        in_jpg = glob.glob(in_dir)
        print(in_jpg)
        os.makedirs("./face/"+ os.path.basename(img_lists[lists]), exist_ok=True)
        for num in in_jpg:
            image = imread(num)
            print(num)
            if image is None:
                print("Not open:", num)
                continue

            image_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")
            # 顔認識の実行
            face_list = cascade.detectMultiScale(image_gs, scaleFactor=1.1, minNeighbors=2, minSize=(64, 64))
            # 顔が１つ以上検出された時
            if len(face_list) > 0:
                for rect in face_list:
                    x, y, width, height = rect
                    image = image[rect[1]:rect[1] + rect[3], rect[0]:rect[0] + rect[2]]
                    if image.shape[0] < 64:
                        continue
                    image = cv2.resize(image, (64, 64))
                    # 保存
                    fileName = "./face/"+ os.path.basename(img_lists[lists])+"/"+str(save_num)+".png"
                    save_num+=1
                    imwrite(str(fileName), image)
                    print(fileName)
                    print(img_lists[lists]+"/"+str(save_num)+".pngを保存しました.")
            # 顔が検出されなかった時
            else:
                print("no face")
                continue
            print(image.shape)


``` 
スクレイピングした画像はおなじ階層に以下のようなフォルダ構成で保存されます。
>face/  
>├ 検索ワード1/  
>  │　       　　├ 1.png  
>  │　       　　├ 2.png  
>  │　       　　├ ...  
>  │　       
>├ 検索ワード2/  
>  │　       　　├ 1.png  
>  │　       　　├ 2.png  
>  │　       　　├ ...  　       
>...
  
![顔認識したフォルダ](./2.png)
こういった感じです。 
  
ここで手作業で、  
"分類したい人ではない画像"、"顔ではない部分の画像"、"学習に悪影響を及ぼしそうな画像"  
を消去していきます。（苦行）  
  
  
700枚近くあった画像がこの時点で150～250枚程度まで減っています。  

顔認識の時点でかなりの枚数が削られているので、スクレイピングの時点で高画質な画像を持ってこれるようにすれば改善するかもしれません。  
それをするには画像それぞれのリンク先に飛ぶなりしてオリジナルの画像を持ってこる必要がありそうなので、いつか挑戦してみたいです。

## 画像仕分け
テストデータと教師データのもととなる画像を分けるために画像を一定の割合でランダムにフォルダ分けするコードです。
  
  
こちらも、日本語パス対策とGUIでのフォルダ指定ができるようにしています。
  
顔をトリミングした画像フォルダがあるフォルダを指定します。（今回はfaceフォルダ）
``` python:deviede_test_train.py
import shutil
import random
import glob
import os
import tkinter
from tkinter import filedialog
os.makedirs("../test", exist_ok=True)

# フォルダ指定
def dirdialog_clicked():
    root = tkinter.Tk()
    root.withdraw()
    iDir = os.path.abspath(os.path.dirname(__file__))
    iDirPath = filedialog.askdirectory(initialdir = iDir)
    root.destroy()
    return iDirPath


if __name__ == '__main__':
    # 画像フォルダ指定
    image_dir = dirdialog_clicked()
    img_lists = os.listdir(path=image_dir)
    for lists in range(len(img_lists)):
        in_dir = image_dir +"/"+img_lists[lists]+"/*"
        in_jpg=glob.glob(in_dir)
        img_file_name_list=os.listdir(image_dir +"/"+img_lists[lists]+"/")
        #img_file_name_listをシャッフル、そのうち2割をtest_imageディテクトリに入れる
        random.shuffle(in_jpg)
        os.makedirs('./test/' + os.path.basename(img_lists[lists]), exist_ok=True)
        for t in range(len(in_jpg)//5):
            shutil.move(str(in_jpg[t]), "./test/"+
``` 
これはデータセット全体を5分割し、うち1つをtestフォルダに送るものです。  
成功すると、画像の8割がfaceフォルダに残り、2割がtestフォルダに送られています。
![faceとtest](./3.png)  

## 学習データの水増し
ほかのコードと同じ変更に加えて、水増しのための画像加工に"左右反転"も加えました。  

参考記事では、人の顔を左右反転するのもなあ、といった感じで反転はしないこととしていましたが、アイドルの自撮り画像は使用してるアプリの影響で左右反転されてるものも少なからずあるのと、教師データをめちゃくちゃに増やしてみたいという野望があったのでぼくは反転を取り入れることにしました。  
      
...と思ったのですが、回転させたオリジナルの画像、ぼかした画像、閾値処理した画像すべてに反転をかけて学習させると過学習を起こしてしまったので、オリジナルを反転させたモノだけを追加しました。

なので、回転（3）×(オリジナル、ぼかし、閾値処理)＋反転で教師データが10倍になります。  
  
  
コードを実行する際は、水増ししたい画像が入っているフォルダ（今回はface）を選択します。
  
``` python:inflation.py
import os
import cv2
import glob
from scipy import ndimage
import tkinter
from tkinter import filedialog
import numpy as np
"""
faceディレクトリから画像を読み込んで回転、ぼかし、閾値処理をしてtrainディレクトリに保存する.
"""

# フォルダ指定
def dirdialog_clicked():
    root = tkinter.Tk()
    root.withdraw()
    iDir = os.path.abspath(os.path.dirname(__file__))
    iDirPath = filedialog.askdirectory(initialdir = iDir)
    root.destroy()
    return iDirPath

# パスに日本語が含まれる場合の対策
# np.profileとcv2.imdecodeに分解した
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None
#cv2.imencode + np.ndarray.tofile に分解
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)

        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False



if __name__ == '__main__':
    os.makedirs("./train", exist_ok=True)

    # 画像フォルダ指定
    image_dir = dirdialog_clicked()
    img_lists = os.listdir(path=image_dir)

    for lists in range(len(img_lists)):
        in_dir = image_dir +"/"+img_lists[lists] + "/*"
        out_dir = "./train/" + img_lists[lists]+"/"
        os.makedirs(out_dir, exist_ok=True)
        in_jpg=glob.glob(in_dir)
        save_num=0
        for i in in_jpg:
            img = imread(i)
            # 左右反転

            img_flip = cv2.flip(img, 1)
            fileName = os.path.join(out_dir, str(save_num) + "_flip.jpg")
            imwrite(str(fileName), img_flip)
            # 回転
            for ang in [-10,0,10]:
                img_rot = ndimage.rotate(img,ang)
                img_rot = cv2.resize(img_rot,(64,64))
                fileName=os.path.join(out_dir,str(save_num)+"_"+str(ang)+".jpg")
                imwrite(str(fileName),img_rot)
                # 閾値
                img_thr = cv2.threshold(img_rot, 100, 255, cv2.THRESH_TOZERO)[1]
                fileName=os.path.join(out_dir,str(save_num)+"_"+str(ang)+"thr.jpg")
                imwrite(str(fileName),img_thr)
                # ぼかし
                img_filter = cv2.GaussianBlur(img_rot, (5, 5), 0)
                fileName=os.path.join(out_dir,str(save_num)+"_"+str(ang)+"filter.jpg")
                imwrite(str(fileName),img_filter)
            save_num += 1
```
![水増し後](./4.png)
こんな感じになります。  
  
それぞれ1000～2000枚くらいの教師データがtrainフォルダに保存されました。 
  
ただ、人物によってかなり枚数に差が出てしまうのは学習にどんな影響を及ぼしているかわからないので不安です。改善するべきかも？

## 学習
これ以前のコード変更に加えて、分類するクラス数（人数）を好きにできるように書き換えました。  
また、モデルの保存一番最後にしていたために、学習終わって精度のグラフのプロットでエラーが出て、十数分無駄にしてしまったので、モデルの保存を学習した直後にするようにしました。

しかしその辺りは勉強不足なので、クラス数を増やしすぎたりするのは良くなさそうだと思いました。  
  
  
フォルダ選択は、教師データが入っているフォルダ（今回はtrain）を選択します。

  
``` python:learn.py
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tkinter
from tkinter import filedialog

# フォルダ指定
def dirdialog_clicked():
    root = tkinter.Tk()
    root.withdraw()
    iDir = os.path.abspath(os.path.dirname(__file__))
    iDirPath = filedialog.askdirectory(initialdir = iDir)
    root.destroy()
    return iDirPath


image_dir = dirdialog_clicked()
name = os.listdir(path=image_dir)

# パスに日本語が含まれる場合の対策
# np.profileとcv2.imdecodeに分解した
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None
#cv2.imencode + np.ndarray.tofile に分解
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)

        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False


# 教師データのラベル付け
X_train = []
Y_train = []
for i in range(len(name)):
    img_file_name_list=os.listdir("./train/"+name[i])
    print(len(img_file_name_list))
    for j in range(0,len(img_file_name_list)-1):
        n=os.path.join("./train/"+name[i]+"/",img_file_name_list[j])
        img = imread(n)
        b,g,r = cv2.split(img)
        img = cv2.merge([r,g,b])
        X_train.append(img)
        Y_train.append(i)

# テストデータのラベル付け
X_test = [] # 画像データ読み込み
Y_test = [] # ラベル（名前）
for i in range(len(name)):
    img_file_name_list=os.listdir("./test/"+name[i])
    print(len(img_file_name_list))
    for j in range(0,len(img_file_name_list)-1):
        n=os.path.join("./test/"+name[i]+"/",img_file_name_list[j])
        img = imread(n)
        b,g,r = cv2.split(img)
        img = cv2.merge([r,g,b])
        X_test.append(img)
        # ラベルは整数値
        Y_test.append(i)
X_train=np.array(X_train)
X_test=np.array(X_test)

from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.utils.np_utils import to_categorical

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

# モデルの定義
model = Sequential()
model.add(Conv2D(input_shape=(64, 64, 3), filters=32,kernel_size=(3, 3),
                 strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation('sigmoid'))
model.add(Dense(len(name))) # 人数決めるところ
model.add(Activation('softmax'))

# コンパイル
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 学習
history = model.fit(X_train, y_train, batch_size=32,
                    epochs=50, verbose=1, validation_data=(X_test, y_test))

#モデルを保存
model.save("my_model.h5")

# 汎化制度の評価・表示
score = model.evaluate(X_test, y_test, batch_size=32, verbose=0)
print('validation loss:{0[0]}\nvalidation accuracy:{0[1]}'.format(score))

#acc, val_accのプロット
plt.plot(history.history["accuracy"], label="accuracy", ls="-", marker="o")
print("1")
print(history.history)
plt.plot(history.history["val_accuracy"], label="val_accuracy", ls="-", marker="x")
print(history.history)
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()


```
![学習結果](./5.png)
青いほうが教師データと比較したときの精度、オレンジ色のほうが、テストデータと比較したときの精度です。  
青は、学習に使ったデータとそこからできたモデルを比較してるので、正しく学習できていればオレンジより高くなっているはずです。  
  
  
結果としては、わりといい感じになったと思います  
  
層を増やしたりepoch数を増やしても、精度は大体0.8くらいで落ち着いたので、ここからさらに上げるにはどうすればいいのかいろいろ試してみたいです。  
  


# テスト
学習したモデルを使用して実際に分類してみます。  
  
フォルダ名をもとに分類した後のクラス名を指定したので、例のごとくopencvの関数でいくつかエラーが出、その対策を行いました。  
日本語だとcv2.putTextが使用できず、PILのdraw.textで代用した部分は結構頑張りました。  

  
  
実行すると、GUIでファイル選択を求められるので、分類したい人物の顔が映った画像を選択します。
  
```python:predict.py
import numpy as np
import cv2
from keras.models import load_model
import tkinter
from tkinter import filedialog
import os
from PIL import ImageFont, ImageDraw, Image


# フォルダ指定
def dirdialog_clicked():
    root = tkinter.Tk()
    root.withdraw()
    iDir = os.path.abspath(os.path.dirname(__file__))
    iDirPath = filedialog.askdirectory(initialdir=iDir)
    root.destroy()
    return iDirPath


def select_image():
    root = tkinter.Tk()
    root.withdraw()
    fTyp = [("", "*")]
    iDir = os.path.abspath(os.path.dirname(__file__))
    filePath = filedialog.askopenfilename(filetypes=fTyp, initialdir=iDir)
    root.destroy()
    return filePath


# パスに日本語が含まれる場合の対策
# np.profileとcv2.imdecodeに分解した
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None


# cv2.imencode + np.ndarray.tofile に分解
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)

        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False


def puttext(cv_image, text, point, font_path, font_size, color=(0, 0, 0)):
    font = ImageFont.truetype(font_path, font_size)

    cv_rgb_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(cv_rgb_image)

    draw = ImageDraw.Draw(pil_image)
    #draw.text(point, text, fill=color, font=font)
    draw.text(point, text, fill=color, font=font)

    cv_rgb_result_image = np.asarray(pil_image)
    cv_bgr_result_image = cv2.cvtColor(cv_rgb_result_image, cv2.COLOR_RGB2BGR)

    return cv_bgr_result_image


def detect_face(image):
    # opencvを使って顔抽出
    image_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cascade = cv2.CascadeClassifier("./haarcascade_frontalface_alt.xml")
    # 顔認識の実行
    face_list = cascade.detectMultiScale(image_gs, scaleFactor=1.1, minNeighbors=2, minSize=(64, 64))
    # 顔が１つ以上検出された時
    if len(face_list) > 0:
        for rect in face_list:
            x, y, width, height = rect
            cv2.rectangle(image, tuple(rect[0:2]), tuple(rect[0:2] + rect[2:4]), (255, 0, 0), thickness=3)
            img = image[rect[1]:rect[1] + rect[3], rect[0]:rect[0] + rect[2]]
            if image.shape[0] < 64:
                print("too small")
                continue
            img = cv2.resize(image, (64, 64))
            img = np.expand_dims(img, axis=0)
            name = detect_who(img)
            fontpath = 'C:\Windows\Fonts\meiryo.ttc'
            image = puttext(image, name, (x, y + height + 20), fontpath, 24, (255, 0, 0))
            #cv2.putText(image, name, (x, y + height + 20), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 2)
    # 顔が検出されなかった時
    else:
        print("no face")
    return image


def detect_who(img):
    # image_dir = dirdialog_clicked()
    # name_list = os.listdir(path=image_dir)
    name_list = os.listdir(path="./face")
    # 予測
    name = ""
    print(model.predict(img))
    print(name_list)
    nameNumLabel = np.argmax(model.predict(img))
    print(nameNumLabel)
    for num in range(len(name_list)):
        if nameNumLabel == num:
            name = name_list[num]
            print(name,name_list[num])
    return name


if __name__ == '__main__':
    model = load_model('./my_model.h5')
    image = imread(select_image())
    if image is None:
        print("Not open:")
    whoImage = detect_face(image)

    cv2.imshow("result", whoImage)
    cv2.waitKey(0)

```
こういった感じになりました。
![分類結果](./6.png)  

フォルダ名から情報をとってくるので、faceフォルダが存在しないと動かなくなってしまいます。  txtとか、csvなんかに情報だけおいとくように改善できるなと思いました。  
  
  

## 感想
始めてDeepLeerningをやってみましたが、ちゃんと分類できて驚きました。この前に、カスケード分類器を自作して顔検出することに挑戦したのですが、その際はかなり精度が悪かったので、この結果はかなり感動しました。  
コードは参考記事のものを参考にしましたが、やはり少しでも書き換えようとすると、コードの大部分の理解が必要になってくるので、かなり勉強にはなったと思います。  

自分流にしたことによって生じた不都合な部分も多々あるので、今後に生かせると感じました。
  
    
    
ディープラーニングについても理解できた部分はあると思うのですが、まだまだ不十分さを感じたので、これ以降の制作の際にまとめるつもりです。
  
このようにマークダウンを使ってまとめを作るのも初挑戦で、書き方すら参考記事を参考にしてしまった感がありますが、あとで見返すにも、CNNやろうとしてる友人にひけらかすにもとても便利だと感じたので、極力これからも書いていければと思います。